In [142]:
import pandas as pd
from sklearn import preprocessing
from datetime import datetime

In [143]:
df = pd.read_csv("Data/ticket.csv", delimiter="\t")

#### Removing redundant sections and rows with NaN for CheckinTime

In [144]:
df.drop("TIME SECTION",1, inplace = True)
df.drop("TICKET",1, inplace = True)
df.drop("TICKET NUM",1, inplace = True)
df.drop("TABLE TYPE",1, inplace = True)
df.drop("STATUS",1, inplace = True)
df.drop("CHANNEL",1, inplace = True)
df = df.dropna()

In [149]:
df.head(5)

,REST_ID,TABLE SIZE,CREATE TIME,CHECKINTIME,TIME (HR),TIME (SEC),MONTH,DAY
0,113442,5,2016-01-01 11:09:19.000,2016-01-01 11:16:41,11,559,1,5
1,113442,4,2016-01-01 11:15:45.000,2016-01-01 11:16:40,11,945,1,5
2,113442,2,2016-01-01 11:17:11.000,2016-01-01 11:17:21,11,1031,1,5
3,113442,2,2016-01-01 11:34:41.000,2016-01-01 11:34:53,11,2081,1,5
4,113442,5,2016-01-01 11:51:00.000,2016-01-01 12:00:36,11,3060,1,5


In [148]:
df["TIME (HR)"] = df["CREATE TIME"].apply(lambda a: a.split()[1].split(':')[0])
df["TIME (SEC)"] = df["CREATE TIME"].apply(lambda a: int(int(a.split()[1].split(':')[1])*60 + float(a.split()[1].split(':')[2])))
df["MONTH"] = df["CREATE TIME"].apply(lambda a: int(a.split()[0].split('-')[1]))
df["DAY"] = df["CREATE TIME"].apply(lambda a: datetime.strptime(a, "%Y-%m-%d %H:%M:%S.%f").isoweekday() )